In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Step 1: Utilize VGG16 for Feature Extraction
# Load VGG16 model pre-trained on ImageNet, excluding the top layers
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Freeze the convolutional layers of VGG16
for layer in vgg_base.layers:
    layer.trainable = False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
import zipfile

with zipfile.ZipFile('archive.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/extracted_images')

In [7]:
# Step 2: Data Preprocessing and Augmentation
# Setup data generators for training and validation

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/extracted_images',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

validation_generator = test_datagen.flow_from_directory(
    '/content/extracted_images',
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary'
)

Found 25000 images belonging to 2 classes.
Found 25000 images belonging to 2 classes.


In [47]:
# Step 3: Build and Train the Classification Model
# Add custom fully connected layers on top of the VGG16 base model
model = models.Sequential()

# Ensure you are using only one output from vgg_base.
# If vgg_base has multiple outputs, select the appropriate one using vgg_base.output[index]
model.add(vgg_base)

# Add the Flatten layer
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [49]:
# Step 4: Model Compilation and Training
# Compile the model
model.compile(optimizer=optimizers.SGD(learning_rate=0.001, momentum=0.9),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=100,
    epochs=25,
    validation_data=validation_generator,
    validation_steps=50
)

Epoch 1/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 20s 174ms/step - accuracy: 0.5746 - loss: 0.7112 - val_accuracy: 0.8180 - val_loss: 0.4379
Epoch 2/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 171ms/step - accuracy: 0.7082 - loss: 0.5539 - val_accuracy: 0.8460 - val_loss: 0.4054
Epoch 3/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 163ms/step - accuracy: 0.7511 - loss: 0.5134 - val_accuracy: 0.8450 - val_loss: 0.3715
Epoch 4/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 174ms/step - accuracy: 0.7560 - loss: 0.5102 - val_accuracy: 0.7140 - val_loss: 0.5300
Epoch 5/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 164ms/step - accuracy: 0.7399 - loss: 0.5133 - val_accuracy: 0.8250 - val_loss: 0.3803
Epoch 6/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 164ms/step - accuracy: 0.7500 - loss: 0.5022 - val_accuracy: 0.8600 - val_loss: 0.3491
Epoch 7/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 162ms/step - accuracy: 0.7788 - loss: 0.4617 - val_accuracy: 0.8650 - val_loss: 0.3252
Epoch 8/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 161ms/step - accuracy: 0.7903 - loss: 0

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


100/100 ━━━━━━━━━━━━━━━━━━━━ 9s 88ms/step - accuracy: 0.7750 - loss: 0.4613 - val_accuracy: 0.8770 - val_loss: 0.3099
Epoch 14/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 168ms/step - accuracy: 0.8048 - loss: 0.4167 - val_accuracy: 0.8810 - val_loss: 0.2876
Epoch 15/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 168ms/step - accuracy: 0.8025 - loss: 0.4082 - val_accuracy: 0.8920 - val_loss: 0.2715
Epoch 16/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 168ms/step - accuracy: 0.8092 - loss: 0.4127 - val_accuracy: 0.8700 - val_loss: 0.3177
Epoch 17/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 163ms/step - accuracy: 0.8086 - loss: 0.4282 - val_accuracy: 0.8580 - val_loss: 0.3149
Epoch 18/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 170ms/step - accuracy: 0.8004 - loss: 0.4239 - val_accuracy: 0.8630 - val_loss: 0.2910
Epoch 19/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 17s 166ms/step - accuracy: 0.7760 - loss: 0.4482 - val_accuracy: 0.8740 - val_loss: 0.2849
Epoch 20/25
100/100 ━━━━━━━━━━━━━━━━━━━━ 16s 163ms/step - accuracy: 0.7947 - loss: 0.4220 

In [50]:
# Step 5: Evaluate and Test the Model
# Evaluate the model on the validation data
val_loss, val_acc = model.evaluate(validation_generator, steps=50)
print(f'Validation accuracy: {val_acc}')

50/50 ━━━━━━━━━━━━━━━━━━━━ 3s 59ms/step - accuracy: 0.9043 - loss: 0.2751
Validation accuracy: 0.8939999938011169


In [51]:
# Save the model
model.save('my_model.keras')

In [53]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np


def predict_image(model, image_path):
    """
    Predict the class of a new image using the trained model.

    Parameters:
    model: The trained Keras model used for prediction.
    image_path: Path to the image file to be classified.

    Returns:
    Prints the predicted category and the confidence level.
    """
    try:
        # Load and preprocess the image
        img = image.load_img(image_path, target_size=(150, 150))  # Adjust target size to your model's input size
        img_tensor = image.img_to_array(img)  # Convert image to numpy array
        img_tensor = np.expand_dims(img_tensor, axis=0)  # Add batch dimension
        img_tensor /= 255.0  # Normalize pixel values to [0, 1]

        # Make prediction
        prediction = model.predict(img_tensor)
        confidence = prediction[0][0]  # Get the confidence score (since it's a binary classification)

        # Determine the predicted class and confidence level
        if confidence > 0.5:
            predicted_label = 'Dog'
            confidence_level = confidence
        else:
            predicted_label = 'Cat'
            confidence_level = 1 - confidence  # Confidence for Cat is (1 - confidence)

        # Print the result
        print(f"Predicted category: {predicted_label}")
        print(f"Confidence level: {confidence_level:.2f}")

    except Exception as e:
        print(f"An error occurred: {e}")


In [62]:
print(predict_image(model, '/content/extracted_images/cats/cat.1.jpg'))



1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted category: Cat
Confidence level: 1.00
None
